In [4]:
import sys
#!{sys.executable} -m pip install bs4
#!{sys.executable} -m pip install lxml
from lxml import etree
from bs4 import BeautifulSoup
import requests
import pandas as pd


In [5]:
url = 'https://statisticalatlas.com/place/Illinois/Chicago/Population'
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

#use this to find the city blocks
labels = []
for i in range(1251):
    stringer = str(r'dmHover(evt, 1, 2, ' + str(i)+');')
    labels.append(soup.find(onmouseover = stringer))

    
    
labels= list(filter(None.__ne__, labels))    
tiks = list(filter(lambda x: "/tract/Illinois/Cook-County/" in str(x), labels ))

#tiks is a filtered version of labels


blocklinks = []

for i in tiks:
    blocklinks.append(i["xlink:href"])
 
    
blockers = []
for k in blocklinks:
    blockers.append(str('https://statisticalatlas.com')+str(k))


tracts = []

for s in blockers:
    tracts.append(s.replace('https://statisticalatlas.com/tract/Illinois/Cook-County/', '').replace("/Population", ''))
        
tractpop = []
for i in range(len(blockers)):
    url = blockers[i]
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    popdata = soup.find("div", class_ = "sidebar-info-table clearfix")
    tractpop.append(int(popdata.find("td").contents[0].replace(",","")))
    
    
alldata = pd.DataFrame()
alldata["tracts"] = tracts
alldata["population"] = tractpop

In [13]:
tracts = alldata["tracts"]

for i in range(len(tracts)):
    tracts[i] = "tract: " + tracts[i]
    
tracts
    
    
    

<ipython-input-13-dbfca5bb10fe>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracts[i] = "tract: " + tracts[i]


0      tract: 660500
1      tract: 230500
2      tract: 251700
3      tract: 390200
4      tract: 431301
           ...      
795    tract: 350100
796    tract: 730100
797    tract: 690300
798    tract: 051400
799    tract: 243400
Name: tracts, Length: 800, dtype: object

In [17]:
alldata.to_csv(r'/Users/jakefischer/Divvy_repo/DivvyScrape/tract_populations.csv')
